In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/telecom_sms_data.csv')
data.head()

,Event time,label
0,2023-03-03 14:00:00,1057
1,2023-03-03 14:15:00,1013
2,2023-03-03 14:30:00,193
3,2023-03-03 14:45:00,1
4,2023-03-03 15:00:00,0
